**Código de Escoragem**

Nesta etapa, é desenvolvido o código de escoragem, responsável por atribuir pontuações a novos dados que chegam após o modelo estar em produção. Esse código é o que efetivamente será implantado no ambiente de negócio e executado de acordo com a necessidade da operação, podendo rodar em tempo real (online scoring) ou em lotes (batch scoring).

O objetivo da escoragem é aplicar o modelo treinado sobre dados inéditos, classificando clientes quanto ao risco de churn e gerando as informações necessárias para suportar decisões estratégicas, como campanhas de retenção ou otimização de investimentos em marketing.

**1. Importar Bibliotecas**

In [ ]:
# Bibliotecas necessárias para o projeto
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import itertools
import pickle
from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve, roc_auc_score
from sklearn.model_selection import train_test_split


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


**2. Leitura dos dados a serem escorados**

Nesta etapa, são carregados os dados que passarão pelo processo de escoragem, ou seja, a aplicação do modelo preditivo já treinado para atribuir uma probabilidade de churn a cada cliente. Esses dados podem corresponder a uma nova base de clientes ou ao próprio conjunto de teste tratado. A leitura é feita a partir de um arquivo (como .csv) e resulta em um DataFrame preparado para receber o mesmo pipeline de pré-processamento utilizado na fase de treino, garantindo consistência e reprodutibilidade antes da aplicação do modelo.

In [ ]:
# Leitura dos dados a serem escorados
datapath = f'/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/04 - Escoragem/WA_Fn-UseC_-Telco-Customer-Churn.csv'

df00 = pd.read_csv(datapath, sep = ',')

df00.shape


(7043, 22)

In [ ]:
df00['Churn'].replace({'Yes': 1, 'No': 0}, inplace=True)


/tmp/ipython-input-3412476009.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df00['Churn'].replace({'Yes': 1, 'No': 0}, inplace=True)
/tmp/ipython-input-3412476009.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df00['Churn'].replace({'Yes': 1, 'No': 0}, inplace=True)


** 3. Separar 70% dos dados para treino e 30% para validação**


Para avaliar o desempenho do modelo de forma justa, a base de dados destinada à escoragem foi dividida em dois subconjuntos: 70% para treino e 30% para validação. Essa divisão garante que o algoritmo seja ajustado em uma parte da base e posteriormente testado em dados que não foram utilizados no treinamento, permitindo medir sua capacidade de generalização e reduzir riscos de *overfitting*.

In [ ]:
# Separar 70% dos dados para treino e 30% para validação
train, test = train_test_split(df00, test_size=0.3, random_state=42)
train.shape,test.shape


((4930, 22), (2113, 22))

**Retirar espaços da variável "TotalCharges"**

In [ ]:
# Alterar "TotalCharges" para float64
train['TotalCharges'] = train['TotalCharges'].replace(' ', pd.NA)
test['TotalCharges'] = test['TotalCharges'].replace(' ', pd.NA)


In [ ]:
train.head()

Unnamed: 0  customerID  gender  SeniorCitizen Partner Dependents  \
1695        1695  4906-ZHGPK    Male              0     Yes        Yes   
1095        1095  7439-DKZTW    Male              0      No         No   
3889        3889  2592-HODOV    Male              0      No         No   
3667        3667  7826-VVKWT  Female              1     Yes        Yes   
2902        2902  1658-TJVOA  Female              1      No         No   

      tenure PhoneService MultipleLines InternetService  ...  \
1695      54          Yes           Yes             DSL  ...   
1095       1          Yes            No     Fiber optic  ...   
3889      13          Yes            No              No  ...   
3667      24          Yes            No     Fiber optic  ...   
2902       6          Yes            No     Fiber optic  ...   

         DeviceProtection          TechSupport          StreamingTV  \
1695                   No                  Yes                  Yes   
1095                   No                   No                  Yes   
3889  No internet service  No internet service  No internet service   
3667                  Yes                   No                  Yes   
2902                   No                   No                   No   

          StreamingMovies        Contract PaperlessBilling  \
1695                   No        One year              Yes   
1095                   No  Month-to-month              Yes   
3889  No internet service        One year               No   
3667                  Yes        Two year              Yes   
2902                   No  Month-to-month              Yes   

                PaymentMethod MonthlyCharges TotalCharges  Churn  
1695         Electronic check          70.70         3770      0  
1095         Electronic check          80.55        80.55      0  
3889  Credit card (automatic)          19.30       259.65      0  
3667         Electronic check          96.55      2263.45      0  
2902         Electronic check          74.10        450.9      0  

[5 rows x 22 columns]

In [ ]:
test.head()

Unnamed: 0  customerID  gender  SeniorCitizen Partner Dependents  \
185          185  1024-GUALD  Female              0     Yes         No   
2715        2715  0484-JPBRU    Male              0      No         No   
3825        3825  3620-EHIMZ  Female              0     Yes        Yes   
1807        1807  6910-HADCM  Female              0      No         No   
132          132  8587-XYZSF    Male              0      No         No   

      tenure PhoneService     MultipleLines InternetService  ...  \
185        1           No  No phone service             DSL  ...   
2715      41          Yes               Yes              No  ...   
3825      52          Yes                No              No  ...   
1807       1          Yes                No     Fiber optic  ...   
132       67          Yes                No             DSL  ...   

         DeviceProtection          TechSupport          StreamingTV  \
185                    No                   No                   No   
2715  No internet service  No internet service  No internet service   
3825  No internet service  No internet service  No internet service   
1807                  Yes                   No                   No   
132                    No                  Yes                   No   

          StreamingMovies        Contract PaperlessBilling  \
185                    No  Month-to-month              Yes   
2715  No internet service  Month-to-month              Yes   
3825  No internet service        Two year               No   
1807                   No  Month-to-month               No   
132                    No        Two year               No   

                  PaymentMethod MonthlyCharges TotalCharges  Churn  
185            Electronic check          24.80         24.8      1  
2715  Bank transfer (automatic)          25.25       996.45      0  
3825               Mailed check          19.35       1031.7      0  
1807           Electronic check          76.35        76.35      1  
132   Bank transfer (automatic)          50.55       3260.1      0  

[5 rows x 22 columns]

In [ ]:
# Criando um novo dataframe baseado no original
df_train_01 = train.copy()
df_test_01 = test.copy()


In [ ]:
df_train_01.shape


(4930, 22)

**Carregar lista de variáveis que foram excluidas por excesso de nulos**


In [ ]:
# Carregar lista de variáveis que foram excluidas por execesso de nulos
with open('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/01 - Data Prep/prd_drop_nullvars.pkl', 'rb') as f:
    loaded_drop_null_feat = pickle.load(f)
loaded_drop_null_feat


[]

In [ ]:
# Retirando as variáveis excluídas por nulos
df_train_02 = df_train_01.drop(axis=1,columns=loaded_drop_null_feat)
df_test_02 = df_test_01.drop(axis=1,columns=loaded_drop_null_feat)


In [ ]:
df_test_02.head()


Unnamed: 0  customerID  gender  SeniorCitizen Partner Dependents  \
185          185  1024-GUALD  Female              0     Yes         No   
2715        2715  0484-JPBRU    Male              0      No         No   
3825        3825  3620-EHIMZ  Female              0     Yes        Yes   
1807        1807  6910-HADCM  Female              0      No         No   
132          132  8587-XYZSF    Male              0      No         No   

      tenure PhoneService     MultipleLines InternetService  ...  \
185        1           No  No phone service             DSL  ...   
2715      41          Yes               Yes              No  ...   
3825      52          Yes                No              No  ...   
1807       1          Yes                No     Fiber optic  ...   
132       67          Yes                No             DSL  ...   

         DeviceProtection          TechSupport          StreamingTV  \
185                    No                   No                   No   
2715  No internet service  No internet service  No internet service   
3825  No internet service  No internet service  No internet service   
1807                  Yes                   No                   No   
132                    No                  Yes                   No   

          StreamingMovies        Contract PaperlessBilling  \
185                    No  Month-to-month              Yes   
2715  No internet service  Month-to-month              Yes   
3825  No internet service        Two year               No   
1807                   No  Month-to-month               No   
132                    No        Two year               No   

                  PaymentMethod MonthlyCharges TotalCharges  Churn  
185            Electronic check          24.80         24.8      1  
2715  Bank transfer (automatic)          25.25       996.45      0  
3825               Mailed check          19.35       1031.7      0  
1807           Electronic check          76.35        76.35      1  
132   Bank transfer (automatic)          50.55       3260.1      0  

[5 rows x 22 columns]

**Retirar 'Unnamed: 0','customerID' e 'Churn' das tabelas (para escoragem não é necessário e em produção não teremos target)**

In [ ]:
# Retirar ID e Target das tabelas (para escoragem não é necessário e em produção não teremos target)
df_train_02 = df_train_02.drop(axis=1, columns=['Unnamed: 0','customerID','Churn'])
df_test_02 = df_test_02.drop(axis=1, columns=['Unnamed: 0','customerID','Churn'])

df_train_02.shape,df_test_02.shape


((4930, 19), (2113, 19))

In [ ]:
df_train_02.head()


gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
1695    Male              0     Yes        Yes      54          Yes   
1095    Male              0      No         No       1          Yes   
3889    Male              0      No         No      13          Yes   
3667  Female              1     Yes        Yes      24          Yes   
2902  Female              1      No         No       6          Yes   

     MultipleLines InternetService       OnlineSecurity         OnlineBackup  \
1695           Yes             DSL                   No                  Yes   
1095            No     Fiber optic                   No                   No   
3889            No              No  No internet service  No internet service   
3667            No     Fiber optic                   No                   No   
2902            No     Fiber optic                  Yes                   No   

         DeviceProtection          TechSupport          StreamingTV  \
1695                   No                  Yes                  Yes   
1095                   No                   No                  Yes   
3889  No internet service  No internet service  No internet service   
3667                  Yes                   No                  Yes   
2902                   No                   No                   No   

          StreamingMovies        Contract PaperlessBilling  \
1695                   No        One year              Yes   
1095                   No  Month-to-month              Yes   
3889  No internet service        One year               No   
3667                  Yes        Two year              Yes   
2902                   No  Month-to-month              Yes   

                PaymentMethod  MonthlyCharges TotalCharges  
1695         Electronic check           70.70         3770  
1095         Electronic check           80.55        80.55  
3889  Credit card (automatic)           19.30       259.65  
3667         Electronic check           96.55      2263.45  
2902         Electronic check           74.10        450.9

**Carregar as modas das variaveis da base de treino - para substituir nulos**

In [ ]:
# Carregar as modas das variaveis da base de treino
with open('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/01 - Data Prep/prd_fillna.pkl', 'rb') as f:
  loaded_modes = pickle.load(f)

def fill_nulls_with_loaded_modes(dataframe, loaded_modes):
    # Obtém as colunas categóricas com valores nulos
    categorical_columns_with_nulls = dataframe.select_dtypes(include='object').columns[dataframe.select_dtypes(include='object').isnull().any()].tolist()

    # Preenche os valores nulos com os modos carregados
    for column in categorical_columns_with_nulls:
        if column in loaded_modes:
            mode_column = loaded_modes[column]
            dataframe[column].fillna(mode_column, inplace=True)

    return dataframe

# Aplicar imputação de nulos na base de treino
df_train_02 = fill_nulls_with_loaded_modes(df_train_02,loaded_modes)

# Aplicar imputação de nulos na base de teste
df_test_02 = fill_nulls_with_loaded_modes(df_test_02,loaded_modes)


df_test_02.head()


gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
185   Female              0     Yes         No       1           No   
2715    Male              0      No         No      41          Yes   
3825  Female              0     Yes        Yes      52          Yes   
1807  Female              0      No         No       1          Yes   
132     Male              0      No         No      67          Yes   

         MultipleLines InternetService       OnlineSecurity  \
185   No phone service             DSL                   No   
2715               Yes              No  No internet service   
3825                No              No  No internet service   
1807                No     Fiber optic                   No   
132                 No             DSL                   No   

             OnlineBackup     DeviceProtection          TechSupport  \
185                    No                   No                   No   
2715  No internet service  No internet service  No internet service   
3825  No internet service  No internet service  No internet service   
1807                   No                  Yes                   No   
132                    No                   No                  Yes   

              StreamingTV      StreamingMovies        Contract  \
185                    No                   No  Month-to-month   
2715  No internet service  No internet service  Month-to-month   
3825  No internet service  No internet service        Two year   
1807                   No                   No  Month-to-month   
132                    No                   No        Two year   

     PaperlessBilling              PaymentMethod  MonthlyCharges TotalCharges  
185               Yes           Electronic check           24.80         24.8  
2715              Yes  Bank transfer (automatic)           25.25       996.45  
3825               No               Mailed check           19.35       1031.7  
1807               No           Electronic check           76.35        76.35  
132                No  Bank transfer (automatic)           50.55       3260.1

**Transformar variável TotalCharge em float64**


In [ ]:
# **Transformar variável TotalCharge em float64**
df_train_02['TotalCharges'] = pd.to_numeric(df_train_02['TotalCharges'], errors='coerce')
df_test_02['TotalCharges'] = pd.to_numeric(df_test_02['TotalCharges'], errors='coerce')



**Carregar os encoders e a lista de colunas**


**Label Encoder**

In [ ]:
# Carregar os encoders e a lista de colunas
with open('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/01 - Data Prep/prd_labelenc.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

loaded_encoders = loaded_data['encoders']
loaded_columns = loaded_data['columns']

# Aplicando para base de treino
for col in loaded_columns:
    if col in loaded_encoders:
        # Transforma a coluna usando o encoder carregado
        df_train_02[col] = loaded_encoders[col].transform(df_train_02[col])

# Aplicando para base de teste
for col in loaded_columns:
    if col in loaded_encoders:
        # Transforma a coluna usando o encoder carregado
        df_test_02[col] = loaded_encoders[col].transform(df_test_02[col])


**OneHot Encoder**

In [ ]:
# Carregar o encoder e a lista de colunas
with open('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/01 - Data Prep/prd_onehotenc.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

loaded_encoder = loaded_data['encoder']
loaded_columns = loaded_data['columns']

# Aplicando para base completa de treino
encoded_data_test = loaded_encoder.transform(df_train_02[loaded_columns])
encoded_cols_test = loaded_encoder.get_feature_names_out(loaded_columns)
encoded_df_test = pd.DataFrame(encoded_data_test, columns=encoded_cols_test, index=df_train_02.index)
df_train_03 = pd.concat([df_train_02.drop(loaded_columns, axis=1), encoded_df_test], axis=1)

# Aplicando para base de teste
encoded_data_test = loaded_encoder.transform(df_test_02[loaded_columns])
encoded_cols_test = loaded_encoder.get_feature_names_out(loaded_columns)
encoded_df_test = pd.DataFrame(encoded_data_test, columns=encoded_cols_test, index=df_test_02.index)
df_test_03 = pd.concat([df_test_02.drop(loaded_columns, axis=1), encoded_df_test], axis=1)


In [ ]:
df_train_03.head()


SeniorCitizen  tenure  MonthlyCharges  TotalCharges  gender_Male  \
1695              0      54           70.70       3770.00          1.0   
1095              0       1           80.55         80.55          1.0   
3889              0      13           19.30        259.65          1.0   
3667              1      24           96.55       2263.45          0.0   
2902              1       6           74.10        450.90          0.0   

      Partner_Yes  Dependents_Yes  PhoneService_Yes  \
1695          1.0             1.0               1.0   
1095          0.0             0.0               1.0   
3889          0.0             0.0               1.0   
3667          1.0             1.0               1.0   
2902          0.0             0.0               1.0   

      MultipleLines_No phone service  MultipleLines_Yes  ...  \
1695                             0.0                1.0  ...   
1095                             0.0                0.0  ...   
3889                             0.0                0.0  ...   
3667                             0.0                0.0  ...   
2902                             0.0                0.0  ...   

      StreamingTV_No internet service  StreamingTV_Yes  \
1695                              0.0              1.0   
1095                              0.0              1.0   
3889                              1.0              0.0   
3667                              0.0              1.0   
2902                              0.0              0.0   

      StreamingMovies_No internet service  StreamingMovies_Yes  \
1695                                  0.0                  0.0   
1095                                  0.0                  0.0   
3889                                  1.0                  0.0   
3667                                  0.0                  1.0   
2902                                  0.0                  0.0   

      Contract_One year  Contract_Two year  PaperlessBilling_Yes  \
1695                1.0                0.0                   1.0   
1095                0.0                0.0                   1.0   
3889                1.0                0.0                   0.0   
3667                0.0                1.0                   1.0   
2902                0.0                0.0                   1.0   

      PaymentMethod_Credit card (automatic)  PaymentMethod_Electronic check  \
1695                                    0.0                             1.0   
1095                                    0.0                             1.0   
3889                                    1.0                             0.0   
3667                                    0.0                             1.0   
2902                                    0.0                             1.0   

      PaymentMethod_Mailed check  
1695                         0.0  
1095                         0.0  
3889                         0.0  
3667                         0.0  
2902                         0.0  

[5 rows x 30 columns]

In [ ]:
df_train_04 = df_train_03
df_test_04 = df_test_03


**Carregar o scaler**

In [ ]:
# Carregar o scaler
with open('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/01 - Data Prep/prd_scaler.pkl', 'rb') as f:
    loaded_scaler = pickle.load(f)

# Aplicando na base de treino
df_train_04_scaled = loaded_scaler.transform(df_train_04)
df_train_05 = pd.DataFrame(df_train_04_scaled, columns=df_train_04.columns, index=df_train_04.index)

# Aplicando na base de teste
df_test_04s = loaded_scaler.transform(df_test_04)
df_test_05 = pd.DataFrame(df_test_04s, columns=df_test_04.columns, index=df_test_04.index)
df_test_05.head()


SeniorCitizen    tenure  MonthlyCharges  TotalCharges  gender_Male  \
185       -0.436831 -1.284263       -1.327058     -0.998679    -1.032999   
2715      -0.436831  0.349957       -1.312127     -0.569780     0.968056   
3825      -0.436831  0.799367       -1.507892     -0.554221    -1.032999   
1807      -0.436831 -1.284263        0.383397     -0.975924    -1.032999   
132       -0.436831  1.412199       -0.472660      0.429422     0.968056   

      Partner_Yes  Dependents_Yes  PhoneService_Yes  \
185      1.030066       -0.658450         -3.091399   
2715    -0.970811       -0.658450          0.323478   
3825     1.030066        1.518718          0.323478   
1807    -0.970811       -0.658450          0.323478   
132     -0.970811       -0.658450          0.323478   

      MultipleLines_No phone service  MultipleLines_Yes  ...  \
185                         3.091399          -0.867153  ...   
2715                       -0.323478           1.153199  ...   
3825                       -0.323478          -0.867153  ...   
1807                       -0.323478          -0.867153  ...   
132                        -0.323478          -0.867153  ...   

      StreamingTV_No internet service  StreamingTV_Yes  \
185                         -0.528699        -0.793909   
2715                         1.891436        -0.793909   
3825                         1.891436        -0.793909   
1807                        -0.528699        -0.793909   
132                         -0.528699        -0.793909   

      StreamingMovies_No internet service  StreamingMovies_Yes  \
185                             -0.528699            -0.800374   
2715                             1.891436            -0.800374   
3825                             1.891436            -0.800374   
1807                            -0.528699            -0.800374   
132                             -0.528699            -0.800374   

      Contract_One year  Contract_Two year  PaperlessBilling_Yes  \
185           -0.521469          -0.562826              0.841222   
2715          -0.521469          -0.562826              0.841222   
3825          -0.521469           1.776747             -1.188746   
1807          -0.521469          -0.562826             -1.188746   
132           -0.521469           1.776747             -1.188746   

      PaymentMethod_Credit card (automatic)  PaymentMethod_Electronic check  \
185                               -0.524928                        1.399708   
2715                              -0.524928                       -0.714435   
3825                              -0.524928                       -0.714435   
1807                              -0.524928                        1.399708   
132                               -0.524928                       -0.714435   

      PaymentMethod_Mailed check  
185                    -0.539991  
2715                   -0.539991  
3825                    1.851883  
1807                   -0.539991  
132                    -0.539991  

[5 rows x 30 columns]

**Carregar lista de variáveis que passaram pelo Feature Sepection (utilizadas no treinamento do modelo**

In [ ]:
# Carregar lista de variáveis que passaram pelo Feature Sepection (utilizadas no treinamento do modelo)
with open('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/02 - Feature Selection/prd_list_features.pkl', 'rb') as f:
    loaded_features = pickle.load(f)

loaded_features


['SeniorCitizen',
 'tenure',
 'MonthlyCharges',
 'TotalCharges',
 'gender_Male',
 'Partner_Yes',
 'Dependents_Yes',
 'MultipleLines_Yes',
 'InternetService_Fiber optic',
 'OnlineSecurity_Yes',
 'OnlineBackup_Yes',
 'TechSupport_Yes',
 'Contract_One year',
 'Contract_Two year',
 'PaperlessBilling_Yes',
 'PaymentMethod_Electronic check']

In [ ]:
abt_train = df_train_05[loaded_features]
abt_test = df_test_05[loaded_features]


In [ ]:
abt_train.head()


SeniorCitizen    tenure  MonthlyCharges  TotalCharges  gender_Male  \
1695      -0.436831  0.881078        0.195927      0.654499     0.968056   
1095      -0.436831 -1.284263        0.522755     -0.974070     0.968056   
3889      -0.436831 -0.793997       -1.509551     -0.895013     0.968056   
3667       2.289215 -0.344587        1.053643     -0.010511    -1.032999   
2902       2.289215 -1.079985        0.308740     -0.810593    -1.032999   

      Partner_Yes  Dependents_Yes  MultipleLines_Yes  \
1695     1.030066        1.518718           1.153199   
1095    -0.970811       -0.658450          -0.867153   
3889    -0.970811       -0.658450          -0.867153   
3667     1.030066        1.518718          -0.867153   
2902    -0.970811       -0.658450          -0.867153   

      InternetService_Fiber optic  OnlineSecurity_Yes  OnlineBackup_Yes  \
1695                    -0.882690           -0.634162          1.370398   
1095                     1.132901           -0.634162         -0.729715   
3889                    -0.882690           -0.634162         -0.729715   
3667                     1.132901           -0.634162         -0.729715   
2902                     1.132901            1.576884         -0.729715   

      TechSupport_Yes  Contract_One year  Contract_Two year  \
1695         1.566008           1.917660          -0.562826   
1095        -0.638566          -0.521469          -0.562826   
3889        -0.638566           1.917660          -0.562826   
3667        -0.638566          -0.521469           1.776747   
2902        -0.638566          -0.521469          -0.562826   

      PaperlessBilling_Yes  PaymentMethod_Electronic check  
1695              0.841222                        1.399708  
1095              0.841222                        1.399708  
3889             -1.188746                       -0.714435  
3667              0.841222                        1.399708  
2902              0.841222                        1.399708

**Carregando modelo Campeão**

In [ ]:
# Carregando modelo campeão
with open('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/03 - Modelagem/best_model_lightgbm.pkl', 'rb') as f:
  loaded_model = pickle.load(f)

loaded_model



LGBMClassifier(force_col_wise=True, max_depth=3, num_leaves=5, reg_alpha=0.1)

**Escorando base de treino e teste**


In [ ]:

# Escorando base de treino

predprob = loaded_model.predict_proba(abt_train)
predict = loaded_model.predict(abt_train)
abt_train['score_1']=predprob[:,1]
abt_train['class']=predict

# Escorando base de teste

predprob = loaded_model.predict_proba(abt_test)
predict = loaded_model.predict(abt_test)
abt_test['score_1']=predprob[:,1]
abt_test['class']=predict


/tmp/ipython-input-2441080022.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abt_train['score_1']=predprob[:,1]
/tmp/ipython-input-2441080022.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abt_train['class']=predict
/tmp/ipython-input-2441080022.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [ ]:
abt_train = abt_train.merge(df_train_01[['Unnamed: 0','customerID','Churn']], left_index=True, right_index=True, how='inner')
abt_test = abt_test.merge(df_test_01[['Unnamed: 0','customerID','Churn']], left_index=True, right_index=True, how='inner')



In [ ]:
abt_train.head()


SeniorCitizen    tenure  MonthlyCharges  TotalCharges  gender_Male  \
1695      -0.436831  0.881078        0.195927      0.654499     0.968056   
1095      -0.436831 -1.284263        0.522755     -0.974070     0.968056   
3889      -0.436831 -0.793997       -1.509551     -0.895013     0.968056   
3667       2.289215 -0.344587        1.053643     -0.010511    -1.032999   
2902       2.289215 -1.079985        0.308740     -0.810593    -1.032999   

      Partner_Yes  Dependents_Yes  MultipleLines_Yes  \
1695     1.030066        1.518718           1.153199   
1095    -0.970811       -0.658450          -0.867153   
3889    -0.970811       -0.658450          -0.867153   
3667     1.030066        1.518718          -0.867153   
2902    -0.970811       -0.658450          -0.867153   

      InternetService_Fiber optic  OnlineSecurity_Yes  ...  TechSupport_Yes  \
1695                    -0.882690           -0.634162  ...         1.566008   
1095                     1.132901           -0.634162  ...        -0.638566   
3889                    -0.882690           -0.634162  ...        -0.638566   
3667                     1.132901           -0.634162  ...        -0.638566   
2902                     1.132901            1.576884  ...        -0.638566   

      Contract_One year  Contract_Two year  PaperlessBilling_Yes  \
1695           1.917660          -0.562826              0.841222   
1095          -0.521469          -0.562826              0.841222   
3889           1.917660          -0.562826             -1.188746   
3667          -0.521469           1.776747              0.841222   
2902          -0.521469          -0.562826              0.841222   

      PaymentMethod_Electronic check   score_1  class  Unnamed: 0  customerID  \
1695                        1.399708  0.085145      0        1695  4906-ZHGPK   
1095                        1.399708  0.832975      1        1095  7439-DKZTW   
3889                       -0.714435  0.026645      0        3889  2592-HODOV   
3667                        1.399708  0.100149      0        3667  7826-VVKWT   
2902                        1.399708  0.509079      1        2902  1658-TJVOA   

     Churn  
1695     0  
1095     0  
3889     0  
3667     0  
2902     0  

[5 rows x 21 columns]

In [ ]:
abt_test.head()


SeniorCitizen    tenure  MonthlyCharges  TotalCharges  gender_Male  \
185       -0.436831 -1.284263       -1.327058     -0.998679    -1.032999   
2715      -0.436831  0.349957       -1.312127     -0.569780     0.968056   
3825      -0.436831  0.799367       -1.507892     -0.554221    -1.032999   
1807      -0.436831 -1.284263        0.383397     -0.975924    -1.032999   
132       -0.436831  1.412199       -0.472660      0.429422     0.968056   

      Partner_Yes  Dependents_Yes  MultipleLines_Yes  \
185      1.030066       -0.658450          -0.867153   
2715    -0.970811       -0.658450           1.153199   
3825     1.030066        1.518718          -0.867153   
1807    -0.970811       -0.658450          -0.867153   
132     -0.970811       -0.658450          -0.867153   

      InternetService_Fiber optic  OnlineSecurity_Yes  ...  TechSupport_Yes  \
185                     -0.882690           -0.634162  ...        -0.638566   
2715                    -0.882690           -0.634162  ...        -0.638566   
3825                    -0.882690           -0.634162  ...        -0.638566   
1807                     1.132901           -0.634162  ...        -0.638566   
132                     -0.882690           -0.634162  ...         1.566008   

      Contract_One year  Contract_Two year  PaperlessBilling_Yes  \
185           -0.521469          -0.562826              0.841222   
2715          -0.521469          -0.562826              0.841222   
3825          -0.521469           1.776747             -1.188746   
1807          -0.521469          -0.562826             -1.188746   
132           -0.521469           1.776747             -1.188746   

      PaymentMethod_Electronic check   score_1  class  Unnamed: 0  customerID  \
185                         1.399708  0.622205      1         185  1024-GUALD   
2715                       -0.714435  0.152131      0        2715  0484-JPBRU   
3825                       -0.714435  0.012364      0        3825  3620-EHIMZ   
1807                        1.399708  0.766203      1        1807  6910-HADCM   
132                        -0.714435  0.022592      0         132  8587-XYZSF   

     Churn  
185      1  
2715     0  
3825     0  
1807     1  
132      0  

[5 rows x 21 columns]

**Salvando como arquivo csv**

In [ ]:
#### Salvando como arquivo csv
abt_train.to_csv('/content/drive/MyDrive/Colab Notebooks/Projetos/Predicao de Churn - Empresa Telecom/abt_scored_002.csv', index=False)


**Conclusão**

Ao final do projeto, o modelo desenvolvido mostrou-se capaz de prever a probabilidade de um cliente vir a cancelar o serviço, fornecendo informações valiosas para a adoção de estratégias preventivas e de retenção antes que o desligamento ocorra.

O desempenho obtido confirma sua eficácia na diferenciação entre clientes mais e menos propensos ao churn. Entre os algoritmos avaliados, o LightGBM apresentou os melhores resultados, destacando-se pela boa ordenação dos scores de probabilidade e pela robustez de sua métrica AUC, o que reforça a confiança em sua capacidade discriminativa.

Outro ponto de destaque foi a redução de falsos positivos, permitindo que os investimentos em ações de retenção sejam direcionados de forma mais eficiente, evitando custos desnecessários com clientes que não apresentam risco real de churn.

Além disso, a consistência observada nas métricas KS, Gini e AUC entre os conjuntos de treino e teste indica ausência de overfitting, garantindo que o modelo seja robusto e generalizável. Dessa forma, o LightGBM se consolida como uma ferramenta confiável para apoiar a tomada de decisões estratégicas voltadas à retenção de clientes.